In [3]:
from pyspark.sql import SparkSession
spark=SparkSession.builder \
.appName('Spark SQL Managed vs External') \
.enableHiveSupport() \
.getOrCreate()

In [5]:
df=spark.read\
.format('csv')\
.option('inferSchema',"True")\
.option('header','True')\
.load('/data/first_100_customers.csv')
df.show(5)

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    false|
+-----------+----------+---------+-----------+-------+-----------------+---------+
only showing top 5 rows



In [6]:
df.createOrReplaceTempView('temp_customers')

In [7]:
spark.sql('select* from temp_customers limit 5').show()

+-----------+----------+---------+-----------+-------+-----------------+---------+
|customer_id|      name|     city|      state|country|registration_date|is_active|
+-----------+----------+---------+-----------+-------+-----------------+---------+
|          0|Customer_0|     Pune|Maharashtra|  India|       2023-06-29|    false|
|          1|Customer_1|Bangalore| Tamil Nadu|  India|       2023-12-07|     true|
|          2|Customer_2|Hyderabad|    Gujarat|  India|       2023-10-27|     true|
|          3|Customer_3|Bangalore|  Karnataka|  India|       2023-10-17|    false|
|          4|Customer_4|Ahmedabad|  Karnataka|  India|       2023-03-14|    false|
+-----------+----------+---------+-----------+-------+-----------------+---------+



In [8]:
spark.sql('show tables').show()

ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used


+---------+--------------+-----------+
|namespace|     tableName|isTemporary|
+---------+--------------+-----------+
|         |temp_customers|       true|
+---------+--------------+-----------+



In [9]:
spark.sql('create table managed_customers as select * from temp_customers')

25/04/16 15:08:03 WARN ResolveSessionCatalog: A Hive serde table will be created as there is no table provider specified. You can set spark.sql.legacy.createHiveTableByDefault to false so that native data source table will be created instead.
25/04/16 15:08:03 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [10]:
spark.sql('describe extended managed_customers').show(truncate=False)

+----------------------------+----------------------------------+-------+
|col_name                    |data_type                         |comment|
+----------------------------+----------------------------------+-------+
|customer_id                 |int                               |NULL   |
|name                        |string                            |NULL   |
|city                        |string                            |NULL   |
|state                       |string                            |NULL   |
|country                     |string                            |NULL   |
|registration_date           |date                              |NULL   |
|is_active                   |boolean                           |NULL   |
|                            |                                  |       |
|# Detailed Table Information|                                  |       |
|Catalog                     |spark_catalog                     |       |
|Database                    |default 

In [11]:
!hadoop fs -ls /user/hive/warehouse/

Found 1 items
drwxr-xr-x   - root hadoop          0 2025-04-16 15:08 /user/hive/warehouse/managed_customers


## External Table

In [13]:
!hadoop fs -head /data/external_data/first_100_customers.csv


customer_id,name,city,state,country,registration_date,is_active
0,Customer_0,Pune,Maharashtra,India,2023-06-29,False
1,Customer_1,Bangalore,Tamil Nadu,India,2023-12-07,True
2,Customer_2,Hyderabad,Gujarat,India,2023-10-27,True
3,Customer_3,Bangalore,Karnataka,India,2023-10-17,False
4,Customer_4,Ahmedabad,Karnataka,India,2023-03-14,False
5,Customer_5,Hyderabad,Karnataka,India,2023-07-28,False
6,Customer_6,Pune,Delhi,India,2023-08-29,False
7,Customer_7,Ahmedabad,West Bengal,India,2023-12-28,True
8,Customer_8,Pune,Karnataka,India,2023-06-22,True
9,Customer_9,Mumbai,Telangana,India,2023-01-05,True
10,Customer_10,Pune,Gujarat,India,2023-08-05,True
11,Customer_11,Delhi,West Bengal,India,2023-08-02,False
12,Customer_12,Chennai,Gujarat,India,2023-11-21,False
13,Customer_13,Chennai,Karnataka,India,2023-11-06,True
14,Customer_14,Hyderabad,Tamil Nadu,India,2023-02-07,False
15,Customer_15,Mumbai,Gujarat,India,2023-03-02,True
16,Customer_16,Chennai,Karnataka,India,2023-04-05,False
17,Customer_17,Hyd

In [1]:
spark.sql('''
create external table if not exists external_customers_2(
customer_id INT,
name STRING,
city STRING,
state string,
country string, 
registration_date String,
is_active BOOLEAN
)
using csv
location'/data/external_data'
''')


ivysettings.xml file not found in HIVE_HOME or HIVE_CONF_DIR,/etc/hive/conf.dist/ivysettings.xml will be used
25/04/16 15:12:35 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider csv. Persisting data source table `spark_catalog`.`default`.`external_customers_2` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
25/04/16 15:12:35 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.


DataFrame[]

In [2]:
spark.sql('describe extended external_customers_2').show(truncate=False)

+----------------------------+--------------------------------------------------+-------+
|col_name                    |data_type                                         |comment|
+----------------------------+--------------------------------------------------+-------+
|customer_id                 |int                                               |NULL   |
|name                        |string                                            |NULL   |
|city                        |string                                            |NULL   |
|state                       |string                                            |NULL   |
|country                     |string                                            |NULL   |
|registration_date           |string                                            |NULL   |
|is_active                   |boolean                                           |NULL   |
|                            |                                                  |       |
|# Detaile

In [29]:
spark.sql("DROP TABLE IF EXISTS external_customers")


DataFrame[]

In [3]:
spark.sql('select * from external_customers_2 limit 5').show()

+-----------+---------------+---------+-----------+----------+-----------------+---------+
|customer_id|           name|     city|      state|   country|registration_date|is_active|
+-----------+---------------+---------+-----------+----------+-----------------+---------+
|       NULL|        Kolkata|Telangana|      India|2023-07-11|             True|     NULL|
|     169004|Customer_169004|   Mumbai|West Bengal|     India|       2023-03-02|     true|
|     169005|Customer_169005|Bangalore|  Telangana|     India|       2023-01-16|     true|
|     169006|Customer_169006|    Delhi| Tamil Nadu|     India|       2023-11-30|    false|
|     169007|Customer_169007|    Delhi|West Bengal|     India|       2023-01-29|    false|
+-----------+---------------+---------+-----------+----------+-----------------+---------+



In [4]:
!hadoop fs -ls /user/hive/warehouse/

Found 1 items
drwxr-xr-x   - root hadoop          0 2025-04-16 15:08 /user/hive/warehouse/managed_customers


In [5]:
spark.sql('drop table managed_customers').show()    ## metadata and data both will be deleted

++
||
++
++



In [33]:
##lets talk about external table
spark.sql('drop table external_customers_2').show()

++
||
++
++



In [34]:
!hadoop fs -ls /data/external_data

Found 1 items
-rw-r--r--   2 root hadoop      42451 2025-04-16 14:12 /data/external_data/customers.csv
